Copyright (C) Microsoft Corporation.  
see license file for details 

### Builds docker image for training (experimentation)

Runs basic tests and pushes image to dockerhub  

In [1]:
# Allow multiple displays per cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
%load_ext dotenv

In [3]:
dotenv_file_path='./../not_shared/general.env'

In [4]:
import os

In [5]:
!pwd
!ls -l ./

/datadrive01/prj/AzureChestXRayNoAML/code
total 784
-rw-rw-r-- 1 loginvm022 loginvm022   8483 Nov 17 19:28 000_setup_azure_sa.ipynb
-rw-rw-r-- 1 loginvm022 loginvm022 473180 Nov 17 21:37 010_create_training_docker_image.html
-rwxrwxr-x 1 loginvm022 loginvm022  31427 Nov 17 22:35 010_create_training_docker_image.ipynb
drwxrwxr-x 4 loginvm022 loginvm022   4096 Nov 17 22:24 01_DataPrep
drwxrwxr-x 3 loginvm022 loginvm022   4096 Nov 17 22:34 02_Model
drwxrwxr-x 2 loginvm022 loginvm022   4096 Nov 14 11:53 docker_history
-rwxrwxr-x 1 loginvm022 loginvm022 256521 Nov 16 04:29 edit_python_files.html
-rwxrwxr-x 1 loginvm022 loginvm022   7152 Nov 17 21:41 edit_python_files.ipynb
drwxrwxr-x 3 loginvm022 loginvm022   4096 Nov 15 04:58 src


In [6]:
%dotenv $dotenv_file_path

# Your docker login and image repository name
docker_login = os.getenv('docker_login') 
image_tag = os.getenv('image_tag')
image_version = os.getenv('image_version')

docker_login
image_tag
image_version

'georgedockeraccount'

'/chestxray-no-aml-gpu-root-account'

'1.0.4'

In [7]:
%dotenv
os.getenv('image_version')
dockerfile_name = 'dockerfile'+ '_' + os.getenv('image_version')

'1.0.4'

In [8]:
%%writefile ./../docker/$dockerfile_name

FROM nvidia/cuda:10.0-cudnn7-devel
# FROM nvidia/cuda:9.2-cudnn7-devel
#FROM nvidia/cuda:9.0-cudnn7-devel-ubuntu16.04

MAINTAINER George Iordanescu <ghiordan@microsoft.com>
#based on https://github.com/keras-team/keras/blob/master/docker/Dockerfile

# Install system packages
RUN apt-get update && apt-get install -y --no-install-recommends \
      apt-utils \
      bzip2 \
      g++ \
      git \
      graphviz \
      libgl1-mesa-glx \
      libhdf5-dev \
      openmpi-bin \
      wget \
      ssh \
      rsync && \
    rm -rf /var/lib/apt/lists/*

# Install conda
ENV CONDA_DIR /opt/conda
ENV PATH $CONDA_DIR/bin:$PATH

RUN wget --quiet --no-check-certificate https://repo.continuum.io/miniconda/Miniconda3-4.2.12-Linux-x86_64.sh && \
    echo "c59b3dd3cad550ac7596e0d599b91e75d88826db132e4146030ef471bb434e9a *Miniconda3-4.2.12-Linux-x86_64.sh" | sha256sum -c - && \
    /bin/bash /Miniconda3-4.2.12-Linux-x86_64.sh -f -b -p $CONDA_DIR && \
    rm Miniconda3-4.2.12-Linux-x86_64.sh && \
    echo export PATH=$CONDA_DIR/bin:'$PATH' > /etc/profile.d/conda.sh        

# Install AzCopy        
RUN mkdir -p /tmp/azcopy && mkdir -p /azcopy10 && \
    wget -O /tmp/azcopy/azcopyv10.tar.gz https://aka.ms/downloadazcopy-v10-linux &&  \
    tar -xf /tmp/azcopy/azcopyv10.tar.gz -C /azcopy10 
RUN rm -rf /tmp/azcopy
ENV PATH=/azcopy10/azcopy_linux_amd64_10.3.2:$PATH

# Install Python packages and keras
# ENV NB_USER keras
# ENV NB_UID 1000

# RUN useradd -m -s /bin/bash -N -u $NB_UID $NB_USER && \
#     chown $NB_USER $CONDA_DIR -R && \
#     mkdir -p /src && \
#     chown $NB_USER /src

# USER $NB_USER
RUN mkdir -p /sr

ARG python_version=3.6

RUN conda config --append channels conda-forge
RUN conda install -y python=${python_version} && \
    pip install --upgrade pip && \
    pip install \
      sklearn_pandas \
      tensorflow-gpu==1.14 \
      python-dotenv \
      papermill[azure] &&  \
    conda install \
      bcolz \
      h5py \
      imageio \
      matplotlib \
      mkl \
      numpy \
      nose \
      notebook \
      opencv \
      Pillow \
      pandas \
      pydot \
      pygpu \
      pyyaml \
      scikit-learn \
      scikit-image \
      six \
      theano \
      mkdocs \
      tqdm \
      && \
    conda install pytorch torchvision cudatoolkit=9.2 -c pytorch && \
    git clone git://github.com/keras-team/keras.git /src && pip install -e /src[tests] && \
    pip install git+git://github.com/keras-team/keras.git && \
    pip install git+https://github.com/aleju/imgaug && \
    pip install git+https://www.github.com/keras-team/keras-contrib.git && \
    conda clean -yt

# TF 1.13:cuda 10.0, cudnn 7 
# RUN conda install -y python=${python_version} && \
#     pip install --upgrade pip && \
#     pip install \
#       sklearn_pandas \
#       scikit-image \
#       tensorflow-gpu==1.14 \
#       tensorflow-tensorboard \
#       matplotlib \
#       numpy && \
#     conda install \
#       bcolz \
#       h5py \
#       matplotlib \
#       mkl \
#       nose \
#       notebook \
#       opencv \
#       Pillow \
#       pandas \
#       pydot \
#       pygpu \
#       pyyaml \
#       requests \
#       scikit-learn \
#       six \
#       tqdm \
#       pytorch torchvision cudatoolkit=9.0 -c pytorch && \
#     conda install -c conda-forge imageio && \
#     git clone git://github.com/keras-team/keras.git /src && pip install -e /src[tests] && \
#     pip install git+git://github.com/keras-team/keras.git && \
#     pip install git+https://github.com/aleju/imgaug && \
#     pip install git+https://www.github.com/keras-team/keras-contrib.git && \
#     conda clean -yt     

#ADD theanorc /home/keras/.theanorc

ENV PYTHONPATH='/src/:$PYTHONPATH'

WORKDIR /src

EXPOSE 8888

# CMD jupyter notebook --port=8888 --ip=0.0.0.0


Overwriting ./../docker/dockerfile_1.0.4


In [9]:
%dotenv
docker_image_name = os.getenv('docker_login') + os.getenv('image_tag') + ':' + os.getenv('image_version')
docker_file_location = os.path.join(*(['.', '..','docker', dockerfile_name]))
working_path = '.'

docker_image_name
docker_file_location

'georgedockeraccount/chestxray-no-aml-gpu-root-account:1.0.4'

'./../docker/dockerfile_1.0.4'

In [10]:
!mkdir -p ./docker_history
!cp $docker_file_location ./docker_history/

In [11]:
!echo docker build -t $docker_image_name -f $docker_file_location $working_path
!docker build -t $docker_image_name -f $docker_file_location $working_path

docker build -t georgedockeraccount/chestxray-no-aml-gpu-root-account:1.0.4 -f ./../docker/dockerfile_1.0.4 .
Sending build context to Docker daemon  5.432MB
Step 1/16 : FROM nvidia/cuda:10.0-cudnn7-devel
 ---> eaf2ceb9de1a
Step 2/16 : MAINTAINER George Iordanescu <ghiordan@microsoft.com>
 ---> Using cache
 ---> e7bd4f58179f
Step 3/16 : RUN apt-get update && apt-get install -y --no-install-recommends       apt-utils       bzip2       g++       git       graphviz       libgl1-mesa-glx       libhdf5-dev       openmpi-bin       wget       ssh       rsync &&     rm -rf /var/lib/apt/lists/*
 ---> Using cache
 ---> 21c712aeab39
Step 4/16 : ENV CONDA_DIR /opt/conda
 ---> Using cache
 ---> 94aa18bebe0e
Step 5/16 : ENV PATH $CONDA_DIR/bin:$PATH
 ---> Using cache
 ---> 59b29f3f947f
Step 6/16 : RUN wget --quiet --no-check-certificate https://repo.continuum.io/miniconda/Miniconda3-4.2.12-Linux-x86_64.sh &&     echo "c59b3dd3cad550ac7596e0d599b91e75d88826db132e4146030ef471bb434e9a *Miniconda3-4.2.1

  Created wheel for termcolor: filename=termcolor-1.1.0-cp36-none-any.whl size=5716 sha256=b0e920c5d1e681afc7549dbd1f47730c33498463df2427675579ab0f4659353a
  Stored in directory: /root/.cache/pip/wheels/7c/06/54/bc84598ba1daf8f970247f550b175aaaee85f68b4b0c5ab2c6
  Created wheel for absl-py: filename=absl_py-0.8.1-cp36-none-any.whl size=118623 sha256=e54f09e56cdc9f7b5d91c4024cd3efe9344b637f409da68aaddfce5bacdb2f6f
  Stored in directory: /root/.cache/pip/wheels/a7/15/a0/0a0561549ad11cdc1bc8fa1191a353efd30facf6bfb507aefc
  Created wheel for wrapt: filename=wrapt-1.11.2-cp36-cp36m-linux_x86_64.whl size=71741 sha256=28d294796b00ad17db37de3273a09a54bd5106b91d0c075f16e1be479135dfdf
  Stored in directory: /root/.cache/pip/wheels/d7/de/2e/efa132238792efb6459a96e85916ef8597fcb3d2ae51590dfd
  Created wheel for gast: filename=gast-0.3.2-cp36-none-any.whl size=9814 sha256=e9d8db0ea4b946d4877d634154ba9ba477753780c477386b46801fcb4fe81421
  Stored in directory: /root/.cache/pip/wheels/59/38/c6/234dc39

  Created wheel for pyrsistent: filename=pyrsistent-0.15.5-cp36-cp36m-linux_x86_64.whl size=123852 sha256=1ff4b776824641b19bdb9ac3aca81b2b5f7eb450e6cbc56ab44dc9ef0bb5e7b4
  Stored in directory: /root/.cache/pip/wheels/c4/5a/b2/d22f93864c638398dad50541af906e9838bb22044d2cc3ad3a
Successfully built termcolor absl-py wrapt gast future pyyaml tornado pandocfilters pyrsistent
ERROR: tensorboard 1.14.0 has requirement setuptools>=41.0.0, but you'll have setuptools 36.4.0 which is incompatible.
ERROR: jsonschema 3.1.1 has requirement six>=1.11.0, but you'll have six 1.10.0 which is incompatible.
ERROR: msrest 0.6.10 has requirement requests~=2.16, but you'll have requests 2.14.2 which is incompatible.
ERROR: azure-core 1.0.0 has requirement requests>=2.18.4, but you'll have requests 2.14.2 which is incompatible.
ERROR: azure-datalake-store 0.0.48 has requirement requests>=2.20.0, but you'll have requests 2.14.2 which is incompatible.
  Found existing installation: cryptography 1.8.1
    Uninst

bleach-3.1.0-p 100% |###############################| Time: 0:00:00  60.83 MB/s
distributed-2. 100% |###############################| Time: 0:00:00  61.57 MB/s
importlib_meta 100% |###############################| Time: 0:00:00  51.05 MB/s
jinja2-2.10.3- 100% |###############################| Time: 0:00:00  61.00 MB/s
joblib-0.14.0- 100% |###############################| Time: 0:00:00  62.94 MB/s
jupyter_core-4 100% |###############################| Time: 0:00:00  56.01 MB/s
libopencv-3.4. 100% |###############################| Time: 0:00:00  60.32 MB/s
livereload-2.6 100% |###############################| Time: 0:00:00  31.45 MB/s
markdown-3.1.1 100% |###############################| Time: 0:00:00  57.59 MB/s
networkx-2.4-p 100% |###############################| Time: 0:00:00  37.57 MB/s
nose-1.3.7-py3 100% |###############################| Time: 0:00:00  64.61 MB/s
numpy-1.17.3-p 100% |###############################| Time: 0:00:00  11.26 MB/s
pango-1.42.4-h 100% |###################

  Created wheel for pytest-pep8: filename=pytest_pep8-1.0.6-cp36-none-any.whl size=7575 sha256=06f4a7ed1031098760c2b9ee082825e0fe61c9793c5445296b2dc3bb1fc830b5
  Stored in directory: /root/.cache/pip/wheels/68/34/52/2a6e08e030cef8b3671898f34cecb12750cc4b96ebfe66fe67
  Created wheel for pytest-cache: filename=pytest_cache-1.0-cp36-none-any.whl size=8814 sha256=84be38976f6c16e58528bf303f428d8210ea5d4181eb35bc4022fe14819d2bed
  Stored in directory: /root/.cache/pip/wheels/e4/9e/28/59d0a23800e49808f17920c5922252c661966ca4a36db5ee38
Successfully built pytest-pep8 pytest-cache
  Running setup.py develop for Keras
  Cloning git://github.com/keras-team/keras.git to /tmp/pip-req-build-2qkj237x
  Running command git clone -q git://github.com/keras-team/keras.git /tmp/pip-req-build-2qkj237x
  Created wheel for Keras: filename=Keras-2.3.1-cp36-none-any.whl size=365617 sha256=4ca167044caa4df8c314bb2bccbd7a723d3001a06a507749ef2d89cae8f07183
  Stored in directory: /tmp/pip-ephem-wheel-cache-wkrcduhq/

  Created wheel for imgaug: filename=imgaug-0.3.0-cp36-none-any.whl size=866876 sha256=24685746f0bc21775c798ff091d049f5643ab39d5fe708e5d94434239986c7d3
  Stored in directory: /tmp/pip-ephem-wheel-cache-pfnhl7dt/wheels/9c/f6/aa/41dcf2f29cc1de1da4ad840ef5393514bead64ac9e644260ff
Successfully built imgaug
  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-3nv79qql
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-3nv79qql
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-cp36-none-any.whl size=101271 sha256=1278facffbba8f6cf01f8122952673783669d08e3912ec7d55f29c9d43d865eb
  Stored in directory: /tmp/pip-ephem-wheel-cache-2f4gd5y3/wheels/11/27/c8/4ed56de7b55f4f61244e2dc6ef3cdbaff2692527a2ce6502ba
Successfully built keras-contrib
Cache location: /opt/conda/pkgs
Will remove the following tarballs:

/opt/conda/pkgs
---------------
libgcc-ng-9.1.0-hdf63c60_0.tar.bz2           8.1 MB
pixman-0.38.0-

python-3.6.3-hc9025b9_1.tar.bz2             27.0 MB
cairo-1.14.12-h8948797_3.tar.bz2             1.3 MB
zict-1.0.0-py_0.tar.bz2                       12 KB
conda-4.3.30-py36h5d9f9f4_0.tar.bz2          511 KB
urllib3-1.24.2-py36_0.tar.bz2                153 KB
nose-1.3.7-py36_2.tar.bz2                    214 KB
requests-2.22.0-py36_0.tar.bz2                89 KB
prompt_toolkit-2.0.10-py_0.tar.bz2           227 KB
numexpr-2.7.0-py36h2ffa06c_0.tar.bz2         196 KB
kiwisolver-1.1.0-py36he6710b0_0.tar.bz2       90 KB
ninja-1.9.0-py36hfd86e86_0.conda             1.2 MB
liblapack-3.8.0-11_openblas.tar.bz2           10 KB
pickleshare-0.7.5-py36_0.tar.bz2              13 KB
mistune-0.8.4-py36h7b6447c_0.tar.bz2          54 KB
attrs-19.3.0-py_0.tar.bz2                     39 KB
zlib-1.2.11-h7b6447c_3.tar.bz2               120 KB
pyzmq-18.1.0-py36he6710b0_0.tar.bz2          520 KB
locket-0.2.0-py36_1.tar.bz2                    8 KB
imageio-2.6.1-py36_0.tar.bz2                 3.4 MB
libsodium-1.

Removed binutils_linux-64-2.31.1-h6176602_8.tar.bz2
Removed pyopenssl-17.0.0-py36_0.tar.bz2
Removed libxcb-1.13-h1bed415_1.tar.bz2
Removed python-3.6.2-0.tar.bz2
Removed cloudpickle-1.2.2-py_0.tar.bz2
Removed torchvision-0.4.2-py36_cu92.tar.bz2
Removed livereload-2.6.1-py_0.tar.bz2
Removed libvpx-1.7.0-h439df22_0.tar.bz2
Removed graphviz-2.40.1-h21bd128_2.tar.bz2
Removed python-3.6.3-hc9025b9_1.tar.bz2
Removed cairo-1.14.12-h8948797_3.tar.bz2
Removed zict-1.0.0-py_0.tar.bz2
Removed conda-4.3.30-py36h5d9f9f4_0.tar.bz2
Removed urllib3-1.24.2-py36_0.tar.bz2
Removed nose-1.3.7-py36_2.tar.bz2
Removed requests-2.22.0-py36_0.tar.bz2
Removed prompt_toolkit-2.0.10-py_0.tar.bz2
Removed numexpr-2.7.0-py36h2ffa06c_0.tar.bz2
Removed kiwisolver-1.1.0-py36he6710b0_0.tar.bz2
Removed ninja-1.9.0-py36hfd86e86_0.conda
Removed liblapack-3.8.0-11_openblas.tar.bz2
Removed pickleshare-0.7.5-py36_0.tar.bz2
Removed mistune-0.8.4-py36h7b6447c_0.tar.bz2
Removed attrs-19.3.0-py_0.tar.bz2
Removed zlib-1.2.11-h7b64

In [12]:
cli_command_inside_container=' /bin/bash -c "conda env list; pwd; ls -l /workspace ; ' + \
    'pip3 list | grep tensorflow;' + \
    'python -c \'import tensorflow as tf; ' +\
    'from tensorflow.python.client import device_lib; ' +\
    'print(tf.__version__);' +\
    'gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.1);' +\
    'sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options));' +\
    'local_device_protos = device_lib.list_local_devices(); ' +\
    'print( [x.name for x in local_device_protos if x.device_type == \\"GPU\\"]); ' +\
    'sess.close() ' +\
    '\';' +\
    '"'



cli_command_inside_container

cli_command='docker run --gpus all -it --rm  --name chestxray-no-aml-gpu_docker_container ' + \
'-v $(pwd):/workspace:rw ' + \
docker_image_name + \
cli_command_inside_container

cli_command
!$cli_command

' /bin/bash -c "conda env list; pwd; ls -l /workspace ; pip3 list | grep tensorflow;python -c \'import tensorflow as tf; from tensorflow.python.client import device_lib; print(tf.__version__);gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.1);sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options));local_device_protos = device_lib.list_local_devices(); print( [x.name for x in local_device_protos if x.device_type == \\"GPU\\"]); sess.close() \';"'

'docker run --gpus all -it --rm  --name chestxray-no-aml-gpu_docker_container -v $(pwd):/workspace:rw georgedockeraccount/chestxray-no-aml-gpu-root-account:1.0.4 /bin/bash -c "conda env list; pwd; ls -l /workspace ; pip3 list | grep tensorflow;python -c \'import tensorflow as tf; from tensorflow.python.client import device_lib; print(tf.__version__);gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.1);sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options));local_device_protos = device_lib.list_local_devices(); print( [x.name for x in local_device_protos if x.device_type == \\"GPU\\"]); sess.close() \';"'

# conda environments:
#
base                  *  /opt/conda

/src
total 800
-rw-rw-r-- 1 1003 1003   8483 Nov 17 19:28 000_setup_azure_sa.ipynb
-rw-rw-r-- 1 1003 1003 473180 Nov 17 21:37 010_create_training_docker_image.html
-rwxrwxr-x 1 1003 1003  45174 Nov 17 22:39 010_create_training_docker_image.ipynb
drwxrwxr-x 4 1003 1003   4096 Nov 17 22:24 01_DataPrep
drwxrwxr-x 3 1003 1003   4096 Nov 17 22:34 02_Model
drwxrwxr-x 2 1003 1003   4096 Nov 14 11:53 docker_history
-rwxrwxr-x 1 1003 1003 256521 Nov 16 04:29 edit_python_files.html
-rwxrwxr-x 1 1003 1003   7152 Nov 17 21:41 edit_python_files.ipynb
drwxrwxr-x 3 1003 1003   4096 Nov 15 04:58 src
tensorflow-estimator   1.14.0      
tensorflow-gpu         1.14.0      
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8"

2019-11-17 22:46:30.261837: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1326] Created TensorFlow device (/device:GPU:0 with 1144 MB memory) -> physical GPU (device: 0, name: Tesla K80, pci bus id: 49c2:00:00.0, compute capability: 3.7)
2019-11-17 22:46:30.262352: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1326] Created TensorFlow device (/device:GPU:1 with 1144 MB memory) -> physical GPU (device: 1, name: Tesla K80, pci bus id: 6b42:00:00.0, compute capability: 3.7)
['/device:GPU:0', '/device:GPU:1']


In [13]:
cli_command = 'docker login  --username '+ os.getenv('docker_login') +' --password '+os.getenv('docker_password')
# cli_command
!$cli_command

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/loginvm022/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [14]:
!docker push $docker_image_name

The push refers to repository [docker.io/georgedockeraccount/chestxray-no-aml-gpu-root-account]

b4c46e94: Preparing 
254fe7cf: Preparing 
3d193c30: Preparing 
95f4471d: Preparing 
23b76f45: Preparing 
1ef0ccac: Preparing 
25d66fdc: Preparing 
d5e86d56: Preparing 
fa18caaf: Preparing 
d8b4edec: Preparing 
bdc0b45f: Preparing 
de7ea906: Preparing 
7a55184c: Preparing 
e11ab4a2: Preparing 
13bce073: Preparing 
e43028b3: Preparing 


4c46e94: Pushing   2.29GB/7.092GB

4c46e94: Pushing  4.721GB/7.092GB

4c46e94: Pushed   7.184GB/7.092GBPushing  7.163GB1.0.4: digest: sha256:2351a1badddb0a5a0b789d054c63f016a3f2d3f0552f0fa42bfd23a02f7cb2b4 size: 3891


In [15]:
!jupyter nbconvert --to html 010_create_training_docker_image

[NbConvertApp] Converting notebook 010_create_training_docker_image.ipynb to html
[NbConvertApp] Writing 363464 bytes to 010_create_training_docker_image.html
